In [34]:
import pandas as pd

In [35]:
import io
def load_any(file_bytes, filename):
    if filename.lower().endswith((".xlsx", ".xls")):
        return pd.read_excel(io.BytesIO(file_bytes), header=None, dtype=str)
    return pd.read_csv(io.BytesIO(file_bytes), header=None, dtype=str)



In [36]:
KEYWORDS = ["date", "narr", "desc", "with", "depo", "amount", "value"]

def detect_header(df):
    for i in range(min(40, len(df))):  # scan first 40 rows
        row = " ".join(str(x).lower() for x in df.iloc[i].fillna(""))
        if sum(1 for k in KEYWORDS if k in row) >= 2:
            return i
    return 0  # fallback


In [37]:
def extract_table(file_bytes, filename):
    raw = load_any(file_bytes, filename)
    hdr = detect_header(raw)
    df = pd.read_excel(io.BytesIO(file_bytes), header=hdr, dtype=str) \
         if filename.endswith((".xls",".xlsx")) \
         else pd.read_csv(io.BytesIO(file_bytes), header=hdr, dtype=str)
    return df


In [38]:
def auto_map(df):
    cols = {c.lower(): c for c in df.columns}

    date_col = next((v for k,v in cols.items() if "date" in k), None)
    desc_col = next((v for k,v in cols.items() if "narra" in k or "desc" in k or "part" in k), None)

    # priority: withdrawal/deposit → amount
    withdraw = next((v for k,v in cols.items() if "with" in k), None)
    deposit  = next((v for k,v in cols.items() if "depo" in k), None)

    amount_col = withdraw or deposit or next((v for k,v in cols.items() if "amount" in k), None)

    return date_col, desc_col, withdraw, deposit, amount_col


In [39]:
import re

def safe_float(val):
    if val is None:
        return None

    s = str(val).strip()

    # treat ***** or ---- or NaN as null
    if re.fullmatch(r"[\*\- ]+", s):
        return None

    # remove commas/currency
    s = s.replace(",", "").replace("₹", "").replace("$", "")

    # parentheses negative values: (124.50)
    if s.startswith("(") and s.endswith(")"):
        s = "-" + s[1:-1]

    # final safety check
    if not re.fullmatch(r"-?\d+(\.\d+)?", s):
        return None

    try:
        return float(s)
    except:
        return None


In [40]:
def normalize(df):
    date_col, desc_col, withdraw, deposit, amount_col = auto_map(df)

    def compute_amt(row):
        if withdraw:
            w = safe_float(row.get(withdraw))
            if w is not None:
                return -abs(w)

        if deposit:
            d = safe_float(row.get(deposit))
            if d is not None:
                return abs(d)

        # fallback to amount column
        val = safe_float(row.get(amount_col))
        return val



    df = df.copy()
    df = df.dropna(how="all")  # drop blank rows

    out = pd.DataFrame()
    out["tx_date"] = pd.to_datetime(df[date_col], errors="coerce")
    out["description"] = df[desc_col].astype(str)

    out["amount"] = df.apply(compute_amt, axis=1)

    # drop rows that still have no valid amount or no valid date
    out = out.dropna(subset=["tx_date", "amount"])

    out["direction"] = out["amount"].apply(lambda x: "debit" if x < 0 else "credit")

    return out


In [41]:
def parse_statement(file_bytes, filename):
    df = extract_table(file_bytes, filename)
    df = normalize(df)
    return df[["tx_date", "description", "amount", "direction"]]


In [42]:
with open("Acct Statement_8340_07122025_20.42.57.xls", "rb") as f:
    file_bytes = f.read()


In [43]:
df_final = parse_statement(file_bytes,"Acct Statement_8340_07122025_20.42.57.xls").head(100)

C:\Users\devar\AppData\Local\Temp\ipykernel_6544\4043631916.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  out["tx_date"] = pd.to_datetime(df[date_col], errors="coerce")


In [44]:
df_final.shape

(100, 4)

In [45]:
df_final

,tx_date,description,amount,direction
1,2025-03-04,UPI-SLICE-SLICEBORROW@SLC-NESF0SLCUPI-50936285...,5000.0,credit
2,2025-03-04,UPI-ONECARD-ONECARDBOB@YESPAY-YESB0YESUPI-5093...,-2213.0,debit
3,2025-03-04,UPI-SUMIT ABICHANDANI-SUMITABICHANDANI87@OKHDF...,-2389.0,debit
4,2025-04-04,UPI-RAJVEER SINGH-RAJVEERSINGH66021@OKSBI-IBKL...,-1500.0,debit
5,2025-04-04,UPI-DEVARSH RAJ-7257979798@AXISB-UTIB0001751-5...,-8900.0,debit
...,...,...,...,...
96,2025-05-29,UPI-KUSH ANANDANI-7898569684@YBL-SBIN0030390-...,-62.0,debit
97,2025-05-29,UPI-ZEPTO-ZEPTOONLINE@YBL-YESB0YBLUPI-19430620...,-1771.0,debit
98,2025-05-29,UPI-NPCI BHIM-BHIMCASHBACK@HDFCBANK-HDFC0MERUP...,2.0,credit
99,2025-05-30,UPI-ZEPTO-ZEPTOONLINE@YBL-YESB0YBLUPI-08403173...,-839.0,debit


In [46]:
import mysql.connector

In [47]:
db_configs = {
    "host": "127.0.0.1",
    "user": "root",
    "password": "12345",
    "port": "3306",
    "database": "finance_advisor"
}

In [48]:
conn = mysql.connector.connect(**db_configs)

In [49]:
cursor = conn.cursor(dictionary=True)

In [50]:
QUERY = f"""INSERT INTO transactions (user_id, transaction_date, description_raw,  amount, debit_or_credit, source_file_id) VALUES  (%s, %s, %s, %s, %s, %s)"""

In [51]:
df_final['user'] = 1
df_final['source_file_id'] = 1

In [52]:
df_final['direction'] = df_final['direction'].map({'debit':'D', 'credit':'C'})

In [53]:
df = df_final[['user', 'tx_date', 'description', 'amount', 'direction', 'source_file_id']]

In [54]:
data = [tuple(row) for row in df.itertuples(index=False)]

cursor.executemany(QUERY, data)

In [55]:
conn.commit()
conn.close()

In [67]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("finance-advisor-classifier")

2025/12/17 19:58:31 INFO mlflow.tracking.fluent: Experiment with name 'finance-advisor-classifier' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:C:/Code/CodingJourney/AIJourney/Project_01_ai_finance_advisor/tests/mlruns/2', creation_time=1765981711602, experiment_id='2', last_update_time=1765981711602, lifecycle_stage='active', name='finance-advisor-classifier', tags={}>

In [68]:
import mlflow

# Print connection information
print(f"MLflow Tracking URI: {mlflow.get_tracking_uri()}")
print(f"Active Experiment: {mlflow.get_experiment_by_name('finance-advisor-classifier')}")

# Test logging
with mlflow.start_run():
    mlflow.log_param("test_param", "test_value")
    print("✓ Successfully connected to MLflow!")

MLflow Tracking URI: sqlite:///mlflow.db
Active Experiment: <Experiment: artifact_location='file:C:/Code/CodingJourney/AIJourney/Project_01_ai_finance_advisor/tests/mlruns/2', creation_time=1765981711602, experiment_id='2', last_update_time=1765981711602, lifecycle_stage='active', name='finance-advisor-classifier', tags={}>
✓ Successfully connected to MLflow!


In [24]:
!mlflow server --host 127.0.0.1 --port 8080

^C
